In [1]:
!pip install fastapi
!pip install uvicorn
!pip install nest-asyncio
!git clone https://github.com/FasterDecoding/Medusa.git
!cd Medusa && pip install -e .

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00
Cloning into 'Medusa'...
remote: Enumerating objects: 353, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 353 (delta 98), reused 80 (delta 80), pack-reused 217 (from 1)
Receiving objects: 100% (353/353), 4.89 MiB | 26.63 MiB/s, done.
Resolving deltas: 100% (192/192), done.
Obtaining file:///content/Medusa
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.7/137.7 kB 6.7 MB/s eta 0:00:00
  Preparing metadata 

In [2]:
import os
os.chdir('Medusa')
!pip install pyngrok transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.6 MB/s eta 0:00:00


In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
# need to add 2_ to two files inside model
# look for is_flash_attn_available in modelling_llama_kv.py, modelling_mistral_kv.py
# is_flash_attn_available -> is_flash_attn_2_available to fix the import error.

from fastapi import FastAPI, BackgroundTasks
from pydantic import BaseModel
from typing import List, Optional
import asyncio
from asyncio import Queue
from concurrent.futures import ThreadPoolExecutor
import torch
import numpy as np
from dataclasses import dataclass
import time
from transformers import BitsAndBytesConfig
from medusa.model.medusa_model import MedusaModel
from medusa.model.medusa_choices import mc_sim_7b_63
import os
from pyngrok import ngrok
from huggingface_hub import hf_hub_download
import uvicorn
from queue import Empty

In [5]:
# Cell 3 - Define Models and Classes
class GenerationRequest(BaseModel):
    prompt: str
    max_tokens: int = 256
    temperature: float = 0.7
    top_p: float = 0.9
    stream: bool = False

class GenerationResponse(BaseModel):
    text: str
    generation_time: float

@dataclass
class BatchItem:
    def __init__(self, prompt, max_tokens, temperature, top_p, future):
        self.prompt = prompt
        self.max_tokens = max_tokens
        self.temperature = temperature
        self.top_p = top_p
        self.future = future

In [6]:
# Cell 4 - Initialize FastAPI and Global Variables
app = FastAPI(title="Medusa LLM Service")

BATCH_SIZE = 8
BATCH_TIMEOUT = 0.1  # seconds
request_queue = asyncio.Queue()
executor = ThreadPoolExecutor(max_workers=4)

In [7]:
def initialize_models():
    try:
        # Initialize model with Medusa config
        model = MedusaModel.from_pretrained(
            "FasterDecoding/medusa-vicuna-7b-v1.3",
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True,
            device_map="auto"
        )

        tokenizer = model.get_tokenizer()

        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        return model, tokenizer
    except Exception as e:
        print(f"Model initialization error: {e}")
        raise

In [8]:
# Cell 6 - Initialize Model
print("Initializing model... This may take a few minutes...")
model, tokenizer = initialize_models()
print("Model initialized successfully!")

Initializing model... This may take a few minutes...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/143 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/566 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

medusa_lm_head.pt:   0%|          | 0.00/1.48G [00:00<?, ?B/s]

/content/Medusa/medusa/model/medusa_model.py:156: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  medusa_head_state_dict = torch.load(filename, map_location=model.device)


Model initialized successfully!


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:2400: UserWarning: for 1.1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:2400: UserWarning: for 2.0.linear.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:2400: UserWarning: for 2.0.linear.bias: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you

In [9]:
# Backend Processing Functions
from queue import Queue, Empty

async def process_batch(batch: List[BatchItem]):
    try:
        start_time = time.time()

        for i, item in enumerate(batch):
            with torch.inference_mode():
                input_ids = torch.as_tensor([tokenizer.encode(item.prompt)]).cuda()

                generations = model.medusa_generate(
                    input_ids,
                    max_steps=item.max_tokens,
                    temperature=item.temperature,
                    medusa_choices=mc_sim_7b_63,
                    top_p=item.top_p
                )

                generated_text = ""
                for output in generations:
                    generated_text = output["text"]

                generation_time = time.time() - start_time

                item.future.set_result({
                    "text": generated_text,
                    "generation_time": generation_time
                })

                if torch.cuda.is_available():
                    torch.cuda.empty_cache()

    except Exception as e:
        for item in batch:
            item.future.set_exception(e)

async def batch_processor():
    print("Batch processor started")
    while True:
        batch = []
        try:
            print("Waiting for first item...")
            # first_item = await asyncio.to_thread(request_queue.get)
            first_item = await request_queue.get()
            print("Got first item")
            batch.append(first_item)

            batch_deadline = time.time() + BATCH_TIMEOUT
            while len(batch) < BATCH_SIZE and time.time() < batch_deadline:
                try:
                    item = request_queue.get_nowait()
                    batch.append(item)
                except asyncio.queues.QueueEmpty:  # Use asyncio.queues.QueueEmpty
                    await asyncio.sleep(0.1)
                    continue

            print(f"Processing batch with {len(batch)} items...")
            await process_batch(batch)

        except Exception as e:
            print(f"Error in batch processing: {e}")
            for item in batch:
                item.future.set_exception(e)

In [10]:
# Cell 8 - FastAPI Endpoints
@app.on_event("startup")
async def startup_event():
    asyncio.create_task(batch_processor())

@app.post("/generate", response_model=GenerationResponse)
async def generate(request: GenerationRequest):
    print(f"Received request with prompt: {request.prompt}")
    future = asyncio.Future()

    item = BatchItem(
        prompt=request.prompt,
        max_tokens=request.max_tokens,
        temperature=request.temperature,
        top_p=request.top_p,
        future=future
    )

    print("Putting item in queue...")
    await request_queue.put(item)
    print("Item placed in queue")

    print("Waiting for result...")
    result = await future
    print(f"Got result: {result}")

    return GenerationResponse(**result)

@app.get("/health")
async def health_check():
    return {"status": "healthy"}

<ipython-input-10-dd842fd58458>:2: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")


In [11]:
# Cell 9 - Setup ngrok and Start Server
# Note: You need to sign up for ngrok and get an auth token
!ngrok authtoken 2o8EyZCZfWRWEc1nnKDe0ORtAJ4_6sgfuHGZWaek4cuZL7uS1
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

if __name__ == "__main__":
    config = uvicorn.Config(app, port=8000, log_level="info")
    server = uvicorn.Server(config)
    server.run()

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


INFO:     Started server process [489]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://c395-34-82-224-117.ngrok-free.app" -> "http://localhost:8000"
Batch processor started
Waiting for first item...
Received request with prompt: Donald Trump is
Putting item in queue...
Item placed in queue
Waiting for result...
Got first item
Processing batch with 1 items...
Waiting for first item...
Got result: {'text': 'a president who has been widely criticized for his divisive rhet The 2019-2024 Outlook for Sausage Casings in the United States\nThis study covers the market for sausage casings in the United States. The study includes historical data for 2015, 2016, and 2017, and forecasts for 2', 'generation_time': 26.97541832923889}
INFO:     223.178.85.206:0 - "POST /generate HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [489]


KeyboardInterrupt: 

In [ ]:
# Run this in VSCode while above cell running. Colab doesnt allow multiple cells to run simultaneously
import requests

def test_api():
    # test_prompt = "Once upon a time"
    test_prompt = "Donald Trump is"
    url = "https://c395-34-82-224-117.ngrok-free.app/generate" # Copy paste the ngrok URL here / not a neat way to do it but it works

    try:
        response = requests.post(
            url,
            json={
                "prompt": test_prompt,
                "max_tokens": 50,
                "temperature": 0.7,
                "top_p": 0.9
            },
            timeout=300
        )

        # Print response details for debugging
        print(f"Status Code: {response.status_code}")
        print(f"Response Headers: {response.headers}")
        print(f"Raw Response: {response.text}")

        # Check if response is successful
        response.raise_for_status()

        try:
            return response.json()
        except requests.exceptions.JSONDecodeError as e:
            print(f"Failed to decode JSON: {e}")
            print(f"Response content: {response.content}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None

# Test the API
result = test_api()
if result:
    print("\nParsed JSON Response:", result)
